In [5]:
import os
import time
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Suppresses TensorFlow INFO, WARNING, ERROR

import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib

# Görüntü İşleme
import cv2
import mediapipe as mp

# Makine Öğrenimi (Scikit-learn)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Makine Öğrenimi (TensorFlow)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# --- SABİTLER VE KONFİGÜRASYON ---
DATA_DIR = "/media/osman/Yedek/datasets_signl_language/asl_sign_language/asl_alphabet_train"
LANDMARKS_FILE = "landmarks.csv"
MODEL_FILENAME = "best_sign_language_model.joblib"
LE_FILENAME = "label_encoder.joblib"

# MediaPipe Kurulumu
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
# static_image_mode=False olarak ayarlandı, çünkü statik olarak bir veri seti üzerinde çalışsak da 
# performans ve güvenilirlik için bazen bu daha iyidir.

# ----------------------------------------------------
# 1. LANDMARK ÇIKARMA VEYA YÜKLEME (Önbellekleme)
# ----------------------------------------------------

if os.path.exists(LANDMARKS_FILE):
    print(f"[INFO] {LANDMARKS_FILE} bulundu. Görüntü işleme adımı atlanıyor.")
    df = pd.read_csv(LANDMARKS_FILE)
    
    # Koordinat sütunlarını al
    feature_cols = [col for col in df.columns if col not in ['label']]
    X = df[feature_cols].values
    y = df['label'].values
else:
    print(f"[INFO] {LANDMARKS_FILE} bulunamadı. Görüntüler işleniyor ve landmark'lar çıkarılıyor...")
    
    data = []
    labels = []
    
    start_time = time.time()
    
    for label in os.listdir(DATA_DIR):
        folder_path = os.path.join(DATA_DIR, label)
        if not os.path.isdir(folder_path):
            continue
            
        for img_file in tqdm(os.listdir(folder_path), desc=f"Processing {label}"):
            img_path = os.path.join(folder_path, img_file)
            image = cv2.imread(img_path)
            if image is None:
                continue
                
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image_rgb)

            if results.multi_hand_landmarks:
                landmark_list = []
                # Sadece ilk algılanan eli kullan (max_num_hands=1)
                for lm in results.multi_hand_landmarks[0].landmark:
                    # x, y, z koordinatlarını ekle (toplam 21 * 3 = 63 özellik)
                    landmark_list.extend([lm.x, lm.y, lm.z])
                data.append(landmark_list)
                labels.append(label)

    print(f"\n[INFO] Toplam {len(data)} örnek işlendi.")
    print(f"[INFO] Görüntü İşleme Süresi: {time.time() - start_time:.2f} saniye")
    
    # DataFrame oluştur ve kaydet
    num_features = 63 # 21 landmark * 3 (x, y, z)
    column_names = [f'lm_{i}_{coord}' for i in range(21) for coord in ['x', 'y', 'z']]
    df = pd.DataFrame(data, columns=column_names)
    df['label'] = labels
    
    df.to_csv(LANDMARKS_FILE, index=False)
    print(f"[INFO] Landmark'lar başarıyla {LANDMARKS_FILE} dosyasına kaydedildi.")
    
    X = np.array(data)
    y = np.array(labels)

# ----------------------------------------------------
# 2. VERİ ÖN İŞLEME VE BÖLME
# ----------------------------------------------------

# Etiket Kodlama
le = LabelEncoder()
y_encoded = le.fit_transform(y)
n_classes = len(np.unique(y_encoded))
input_dim = X.shape[1]

print(f"\n[VERİ] Toplam Örnek: {X.shape[0]}, Özellik Sayısı: {input_dim}, Sınıf Sayısı: {n_classes}")

# Veri setini ayırma
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# TensorFlow için one-hot encoding (sadece TF modeli için kullanılacak)
y_train_tf = to_categorical(y_train, num_classes=n_classes)
y_test_tf = to_categorical(y_test, num_classes=n_classes)

# ----------------------------------------------------
# 3. MODELLERİ EĞİTME VE KARŞILAŞTIRMA
# ----------------------------------------------------
results = {}

# --- A) Random Forest Classifier (Rastgele Orman) ---
print("\n--- A) RANDOM FOREST EĞİTİMİ ---")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
start_time = time.time()
rf_model.fit(X_train, y_train)
train_time = time.time() - start_time
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
results['RandomForest'] = {'Accuracy': accuracy_rf, 'Time': train_time, 'Model': rf_model}

print(f"Accuracy (RF): {accuracy_rf:.4f}")
print(f"Eğitim Süresi: {train_time:.2f} saniye")


# --- B) Support Vector Machine (Destek Vektör Makineleri) ---
# Kernel olarak 'rbf' (Radial Basis Function) yaygın kullanılır.
print("\n--- B) SVM EĞİTİMİ (Kernel=rbf) ---")
# C ve gamma gibi hiperparametreler denenebilir.
svm_model = SVC(kernel='rbf', C=10, gamma='auto', random_state=42)
start_time = time.time()
svm_model.fit(X_train, y_train)
train_time = time.time() - start_time
y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
results['SVM'] = {'Accuracy': accuracy_svm, 'Time': train_time, 'Model': svm_model}

print(f"Accuracy (SVM): {accuracy_svm:.4f}")
print(f"Eğitim Süresi: {train_time:.2f} saniye")


# --- C) TensorFlow (Derin Yapay Sinir Ağı - ANN) ---
print("\n--- C) TENSORFLOW ANN EĞİTİMİ ---")

tf_model = Sequential([
    Dense(128, activation='relu', input_shape=(input_dim,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(n_classes, activation='softmax') # Çıkış katmanı, sınıf sayısı kadar nöron ve softmax
])

tf_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

start_time = time.time()
history = tf_model.fit(
    X_train, y_train_tf,
    epochs=50, # Yüksek doğruluk için daha fazla epoch denenebilir
    batch_size=32,
    validation_data=(X_test, y_test_tf),
    verbose=0 # Çıktıyı engelle
)
train_time = time.time() - start_time

loss, accuracy_tf = tf_model.evaluate(X_test, y_test_tf, verbose=0)
results['TensorFlow_ANN'] = {'Accuracy': accuracy_tf, 'Time': train_time, 'Model': tf_model}

print(f"Accuracy (TF ANN): {accuracy_tf:.4f}")
print(f"Eğitim Süresi: {train_time:.2f} saniye")

# ----------------------------------------------------
# 4. BAŞARI KARŞILAŞTIRMASI VE EN İYİ MODELİN SEÇİMİ
# ----------------------------------------------------

print("\n\n--- MODEL BAŞARI VE SÜRE KARŞILAŞTIRMASI ---")
best_model_name = ""
max_accuracy = -1

for name, res in results.items():
    print(f"Model: {name:<15} | Doğruluk: {res['Accuracy']:.4f} | Süre: {res['Time']:.2f}s")
    if res['Accuracy'] > max_accuracy:
        max_accuracy = res['Accuracy']
        best_model_name = name

print(f"\n[SONUÇ] En iyi model: {best_model_name} (Doğruluk: {max_accuracy:.4f})")

# En iyi modeli al
best_model = results[best_model_name]['Model']

# ----------------------------------------------------
# 5. MODELİ VE KODLAYICIYI KAYDETME (Canlı Çalışma İçin)
# ----------------------------------------------------

if best_model_name == 'TensorFlow_ANN':
    # TensorFlow modelini farklı kaydetme mekanizması
    tf_model.save(MODEL_FILENAME.replace(".joblib", ".h5"))
    print(f"[KAYIT] TensorFlow modeli başarıyla {MODEL_FILENAME.replace('.joblib', '.h5')} olarak kaydedildi.")
else:
    # Scikit-learn modellerini kaydetme (Joblib)
    joblib.dump(best_model, MODEL_FILENAME)
    print(f"[KAYIT] {best_model_name} modeli başarıyla {MODEL_FILENAME} olarak kaydedildi.")

# Label Encoder'ı kaydet
joblib.dump(le, LE_FILENAME)
print(f"[KAYIT] Label Encoder başarıyla {LE_FILENAME} olarak kaydedildi.")

# Örnek Sınıflandırma Raporu (En iyi model için)
print(f"\n--- {best_model_name} Modeli Sınıflandırma Raporu ---")
if best_model_name != 'TensorFlow_ANN':
    final_y_pred = best_model.predict(X_test)
    print(classification_report(y_test, final_y_pred, target_names=le.classes_))
else:
    # TF model çıktısını decode et
    final_y_pred_tf = tf_model.predict(X_test)
    final_y_pred = np.argmax(final_y_pred_tf, axis=1)
    print(classification_report(y_test, final_y_pred, target_names=le.classes_))

I0000 00:00:1761250076.623819   22388 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1761250076.625139   22870 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)


[INFO] landmarks.csv bulunamadı. Görüntüler işleniyor ve landmark'lar çıkarılıyor...


W0000 00:00:1761250076.645693   22857 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1761250076.676386   22866 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1761250076.691363   22867 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
Processing M: 100%|█████████████████████████| 3000/3000 [00:57<00:00, 52.53it/s]



[INFO] Toplam 51382 örnek işlendi.
[INFO] Görüntü İşleme Süresi: 1422.70 saniye
[INFO] Landmark'lar başarıyla landmarks.csv dosyasına kaydedildi.

[VERİ] Toplam Örnek: 51382, Özellik Sayısı: 63, Sınıf Sayısı: 28

--- A) RANDOM FOREST EĞİTİMİ ---
Accuracy (RF): 0.9345
Eğitim Süresi: 5.53 saniye

--- B) SVM EĞİTİMİ (Kernel=rbf) ---
Accuracy (SVM): 0.9259
Eğitim Süresi: 17.62 saniye

--- C) TENSORFLOW ANN EĞİTİMİ ---


/home/osman/Masaüstü/environments/sign_language_env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1761251544.934133   22388 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4518 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5
I0000 00:00:1761251546.429712   25212 service.cc:148] XLA service 0x7800a400a2a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1761251546.429841   25212 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
I0000 00:00:1761251546.556936   25212 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00

Accuracy (TF ANN): 0.9389
Eğitim Süresi: 87.77 saniye


--- MODEL BAŞARI VE SÜRE KARŞILAŞTIRMASI ---
Model: RandomForest    | Doğruluk: 0.9345 | Süre: 5.53s
Model: SVM             | Doğruluk: 0.9259 | Süre: 17.62s
Model: TensorFlow_ANN  | Doğruluk: 0.9389 | Süre: 87.77s

[SONUÇ] En iyi model: TensorFlow_ANN (Doğruluk: 0.9389)
[KAYIT] TensorFlow modeli başarıyla best_sign_language_model.h5 olarak kaydedildi.
[KAYIT] Label Encoder başarıyla label_encoder.joblib olarak kaydedildi.

--- TensorFlow_ANN Modeli Sınıflandırma Raporu ---
322/322 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step   
              precision    recall  f1-score   support

           A       0.91      0.94      0.93       355
           B       0.91      0.99      0.95       386
           C       0.97      0.99      0.98       345
           D       0.99      0.91      0.95       473
           E       0.93      0.95      0.94       404
           F       0.97      0.96      0.97       481
           G       0.98      0.98      0.9